In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, recall_score, f1_score
from imblearn.metrics import geometric_mean_score, specificity_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import normalize
import json
import pandas as pd
import numpy as np
import sys, os
import warnings
import datetime
#import pickle
from joblib import dump, load
from pathlib import Path

from sklearn import metrics
from sklearn.metrics import classification_report

from art.attacks.evasion import FastGradientMethod, ZooAttack
from art.estimators.classification import SklearnClassifier

from art.metrics import RobustnessVerificationTreeModelsCliqueMethod, SHAPr


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/art/estimators/certification/__init__.py:28: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


In [2]:


XCols = ['mean', 'sum', 'std']
label = 'label'



In [3]:
#df = pd.read_csv(sys.argv[1], dtype = fieldTypes, parse_dates = ['time_start'])
df = pd.read_csv("/rds/general/user/as9820/home/revist/revisiting-iot-device-identification/data/stats_nov_apr.csv", parse_dates = ['time'])
df.fillna(0, inplace=True)



In [4]:
weeks = [list(range(44,53)), list(range(1,10)), list(range(10,19))]

#trainCon = df['time'] >= pd.Timestamp('2020-03-01')
#trainCon = df['time'] < pd.Timestamp('2020-01-01')
#trainCon = ((df['time'] >= pd.Timestamp('2020-01-01')) & (df['time'] < pd.Timestamp('2020-03-01')))
trainCon = df['time'].dt.isocalendar().week.isin(weeks[0])
#trainCon = df['time'].dt.week.isin([44,45])


#testCon = df['time'] > pd.Timestamp('2020-02-29')
#testCons = pd.date_range('2019-11-01', '2020-05-01', freq = '1W').tolist()
testCons = pd.date_range('2019-11-01', '2020-05-01', freq = '1W').tolist()
#testCons = pd.date_range('2019-11-01', '2019-12-01', freq = '1W').tolist()

#X = dfTrain.iloc[:, 2:5]
#y = dfTrain.iloc[:, 1]
X = df[trainCon][XCols]
y = df[trainCon][label]


In [5]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)


In [7]:
#clf = RandomForestClassifier(n_estimators=20, n_jobs=50)
rfc = RandomForestClassifier(n_jobs=18)
#svc = SVC(verbose=True)
#dtc = DecisionTreeClassifier()
#knn = KNeighborsClassifier(n_jobs=54)
#mv = VotingClassifier(estimators=[('knn', knn), ('dt', dtc), ('rf', rfc), ('svm',svc)], voting='hard', n_jobs=54)

modelFile = Path(f"model/rfc_0_1sec")
if modelFile.is_file():
    print(f"loading model {modelFile}")
    rfc = load(modelFile)
else:
    print(f"start training for weeks {weeks[0]}")
    rfc.fit(X_train.values, y_train)
    print("RFC trained")
    #dump(rfc, f"model/rfc_{sys.argv[2]}")
    dump(rfc, modelFile)


loading model model/rfc_0_1sec


In [21]:
for date in testCons:
    testCon = df['time'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    testDF = df[testCon]

    X = testDF[XCols]
    y = testDF[label]
    
    if len(X) == 0: 
        continue
    
    #y_pred = clf.predict(X)
    y_rfc = rfc.predict(X)
    #print("rfc predict")
    #y_svc = svc.predict(X)
    #print("svc predict")
    #y_dtc = dtc.predict(X)
    #print("dtc predict")
    #y_knn = knn.predict(X)
    #print("knn predict")
    #y_mv = mv.predict(X)
    #print("mv predict")
    
    print(f"{date.week}", metrics.f1_score(y, y_rfc, average='micro'))
    print(classification_report(y,y_rfc))#, labels=devs))

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


44 0.7165006309568348


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

              precision    recall  f1-score   support

           7       0.83      0.49      0.62      6745
           8       0.51      0.89      0.65     36559
           9       0.36      0.02      0.04      9418
          10       0.90      0.91      0.91     12953
          12       0.83      0.38      0.52      7509
          14       0.84      0.82      0.83     11626
          15       0.63      0.50      0.56     23081
          16       0.53      0.49      0.51     66903
          17       0.00      0.00      0.00         0
          18       0.90      0.97      0.93     66743
          19       0.58      0.09      0.15     36295
          20       0.69      0.60      0.64       944
          22       0.87      1.00      0.93     81312
          23       0.51      0.50      0.50      4289
          24       0.72      0.19      0.30     11994
          25       0.84      0.99      0.91     16473
          27       0.98      0.33      0.49      5368
          28       0.47    

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


...

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


17 0.6466845025320451


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

              precision    recall  f1-score   support

           7       0.61      0.29      0.40     24604
           8       0.00      0.00      0.00         0
           9       0.32      0.16      0.22     64662
          10       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          14       0.79      0.64      0.70     67195
          15       0.07      0.29      0.12     18700
          16       0.06      0.81      0.11     14281
          17       0.85      0.34      0.49     23117
          18       0.99      0.97      0.98    403245
          19       1.00      0.93      0.96    220232
          20       0.04      0.00      0.00    145608
          22       0.00      0.00      0.00         0
          23       0.54      0.25      0.34     51621
          24       0.56      0.15      0.23     74235
          25       0.56      0.95      0.71     25054
          27       0.95      0.45      0.61     35077
          28       0.98    

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
advClassifier = SklearnClassifier(rfc)
fgm = FastGradientMethod(estimator=advClassifier)

for date in testCons:
    testCon = df['time_start'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    testDF = df[testCon]

    X = testDF[features]
    y = testDF[label]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, stratify=y)
    
    if len(X) == 0: 
        continue
    X_test_adv=fgm.generate(X_test.values)
    np.savetxt(f"rf1secfgsm-{date.week}.csv", x_test_adv, delimiter=",")


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


EstimatorError: FastGradientMethod requires an estimator derived from <class 'art.estimators.estimator.BaseEstimator'> and <class 'art.estimators.estimator.LossGradientsMixin'>, the provided classifier is an instance of <class 'art.estimators.classification.scikitlearn.ScikitlearnRandomForestClassifier'> and is derived from (<class 'art.estimators.classification.scikitlearn.ScikitlearnClassifier'>,).

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, stratify=y)
X_test = normalize(X=X_test)

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [ ]:
advClassifier = SklearnClassifier(rfc)
rt = RobustnessVerificationTreeModelsCliqueMethod(classifier=advClassifier)
average_bound, verified_error = rt.verify(x=X_test, y=y_test, eps_init=0.3, nb_search_steps=10, max_clique=2, 
                                          max_level=2)

print('Average bound:', average_bound)
print('Verified error at eps:', verified_error)

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


In [26]:
advClassifier = SklearnClassifier(rfc)
zoo = ZooAttack(classifier=advClassifier, confidence=0.0, targeted=False, learning_rate=0.1, max_iter=5,
                    binary_search_steps=10, initial_const=1e-3, abort_early=True, use_resize=False, 
                    use_importance=False, nb_parallel=1, batch_size=1, variable_h=0.2)

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


TypeError: '<' not supported between instances of 'list' and 'int'

In [28]:
for date in testCons:
    testCon = df['time'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    testDF = df[testCon]

    X = testDF[XCols]
    y = testDF[label]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, stratify=y)
    
    if len(X) == 0: 
        continue
    x_test_adv = zoo.generate(x=X_test.values)
   
    np.savetxt(f"rf1sec_2adv-{date.week}.csv", x_test_adv, delimiter=",")

   # Step 7: Evaluate the ART classifier on adversarial test examples
    y_rfc = advClassifier.predict(x_test_adv)
    dist = np.linalg.norm(x_test_adv-X_test)
    print(dist)
    #print(f"{date.week}", metrics.f1_score(y, y_rfc, average='micro'))

ZOO: 100%|██████████| 200/200 [04:18<00:00,  1.29s/it]


4.660847436646648


ZOO: 100%|██████████| 200/200 [04:14<00:00,  1.27s/it]


4.7101071336172025


ZOO: 100%|██████████| 200/200 [04:13<00:00,  1.27s/it]


...

3.27106818329085


ZOO: 100%|██████████| 200/200 [04:17<00:00,  1.29s/it]


3.4279484620392875


ZOO: 100%|██████████| 200/200 [04:12<00:00,  1.26s/it]

3.3719256445073706


In [1]:
advClassifier = SklearnClassifier(rfc)
zoo = ZooAttack(classifier=advClassifier, confidence=0.0, targeted=True, learning_rate=0.1, max_iter=5,
                    binary_search_steps=10, initial_const=1e-3, abort_early=True, use_resize=False, 
                    use_importance=False, nb_parallel=1, batch_size=1, variable_h=0.2)

NameError: name 'SklearnClassifier' is not defined

In [9]:
art_classifier = SklearnClassifier(rfc)
SHAPr_leakage = SHAPr(art_classifier, X_train, y_train, X_test, y_test)
print("Average SHAPr leakage random forest: ", np.average(SHAPr_leakage))
print("Max SHAPr leakage random forest: ", np.max(SHAPr_leakage))


IndexError: index 33 is out of bounds for axis 1 with size 24